TensorFlow __variable__ 是最好的表示被程序操作的shared，persistent state（represent shared, persistent state manipulated by your program.）  
通过计算操作，他们的值可以改变  
不同于tf.Tensor对象，tf.Variable存在于session.run的内容之外

In [2]:
import tensorflow as tf

### Creating a Variable
最好的创建变量的方式是 __tf.get_variable__ 函数，这个函数要求你specify变量的名字，这个名字也可以被其他的复制（other replicas）进入当前的变量，也可以在checkpointing和导出model的时候命名这个变量的值。 __tf.get_variable__ 也可以用来重新使用之前创建的同名变量，让模型可以更加容易的定义重复利用的layers。

In [10]:
# 创建三维的形状为[1,2,3]的tensor，默认的dtype为tf.float32，
# 初始化值可以通过tf.glorot_uniform_initializer来获得随机值    
my_variable = tf.get_variable("my_variable",[1,2,3])
tf.glorot_uniform_initializer()

    
# 自定义dtype，并且对tf.get_variable初始化：
my_int_variable = tf.get_variable("my_init_variable",[1,2,3],dtype=tf.int32,
                                 initializer = tf.zeros_initializer)

# 初始化tf.Variable为特定值tf.Tensor，这时，不应该定义变量的shape
other_variable = tf.get_variable("other_variable", dtype=tf.int32, 
  initializer=tf.constant([23, 42]))

with tf.Session() as sess:
    print(my_variable)
    print(my_int_variable)

<tf.Variable 'my_variable:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'my_init_variable:0' shape=(1, 2, 3) dtype=int32_ref>


### Variable collections
collections： lists of tensor或者时其他对象，例如tf.Variable实例  
<p>
    默认每个 <b>tf.Variable</b>被放置于两个collections中(get placed in the followinf two collections)：<b> *tf.GraphKeys.GLOBAL_VARIABLE</b>--可以对多个devices共享， <b>*tf.GraphKeys.TRAINABLE_VARIABLES</b>--TensorFlow通过计算梯度的变量
</p>

当不想要某个variable是trainable的时候，将它加到<b>tf.GraphKeys.LOCAL_VARIABLES</b> collection当中，以下的例子当中表示怎么样将一个变量（named my_local）添加到这个collection当中  
同样的，你也可以通过 __trainable=False__ 达到目的  
<p>
    自定义collections,可以用任意string明明，不需要显示的创建一个collection, 在创建完一个变量之后，通过调用 <b>tf.add_to_collection</b>将这个变量（或者是其他对象）添加到这个collection中, 以下举例中，将my_local 变量添加到 <b>my_collection_name</b> 当中  
    获取这个collection当中的所有变量（或者其他对象）可以用 <b>tf.get_collection</b>
    </p>

In [15]:
my_local = tf.get_variable("my_local",shape=(),
                           collections=[tf.GraphKeys.LOCAL_VARIABLES])

my_non_trainable = tf.get_variable("my_non_trainable",shape=(),
                                   trainable=False)
tf.add_to_collection("my_collection_name",my_local)
tf.get_collection("my_collection_name")

[<tf.Variable 'my_local:0' shape=() dtype=float32_ref>]

### Device placement
可以将Variable放在特定的device中，例如，将变量v放在第二个GPU当中（用例如下）  
将变量放在正确的device当中在分布设置中特别重要。将变量放在workers instead of parameter servers可能会严重降低training速度，更加严重的是可能会让每个worker自己伪造同一个变量的不相关的副本。  
__tf.train.replica_device_setter__ 可以自动的将参数放在parameter servers，用例如下。

In [ ]:
with tf.device("/device:GPU:1"):
    v = tf.get_variable("v", [1])

cluster_spec = {
    "ps": ["ps0:2222", "ps1:2222"],
    "worker": ["worker0:2222", "worker1:2222", "worker2:2222"]}
with tf.device(tf.train.replica_device_setter(cluster=cluster_spec)):
    v = tf.get_variable("v", shape=[20, 20])  # this variable is placed 
                                            # in the parameter server
                                            # by the replica_device_setter

### Initializing variables
在使用一个变量之前，它必须首先被初始化。如果说在low-level TensorFlow API进行编程（也就是说，显示的自己创造一个图(graph)和会话(sessions)），就必须要显示的初始化这些变量。大多数的high-level框架(frameworks)例如 __tf.contrib.slim__ , __tf.estimator.Estimator__ 和 __Keras__ 都可以自动的在训练模型之前初始化变量。
<p>
    显示初始化在某些时候是十分有用的，例如在从checkpoint处重新载入模型不重新运行潜在的开销比较大的initializers和在分布设置当中随机初始化变量的时候允许determinism。    
    （Explicit initialization is otherwise useful because it allows you not to rerun potentially expensive initializers when reloading a model from a checkpoint as well as allowing determinism when randomly-initialized variables are shared in a distributed setting）
    </p>
<p>
    在train在是之前，可以通过调用 <b>tf.global_variables_initializer()</b> 一步初始化所有的可以trainable变量。 这个函数返回一个operation在 <b>tf.GraphKeys.GLOVAL_VARIABLES</b> 的collection中为所有的初始化变量负责。通过运行这个operation可以初始化所有的变量，用例如下。 
    </p>
    <p>
    当必须要自己对变量初始化的时候，可以运行变量的初始化operation(variable's initializer operation)，用例如下。
    </p>  
    通过 __tf.report_uninitialized_variables()__ 可以获得一个operation，通过运行这个operation，可以得到所有的没有被初始化的变量名称。
    <p>
    默认的 <b> tf.global_variables_initializer </b> 在变量初始化的时候不能够确定变量被初始化的顺序。因此，如果一个变量的初始化依赖于另外一个变量，则可能会报错。    </p>
    <p>
    所以在任何时候需要使用一个变量，但是并非所有于这个变量有关的变量都被初始化的时候，最好似乎用 <b>variable.initialized_value()</b> 而不是使用 <b>variable</b>,用例如下。  
    (Any time you use the value of a variable in a context in which not all variables are initialized (say, if you use a variable's value while initializing another variable), it is best to use variable.initialized_value() instead of variable)
    </p>   

In [ ]:
## initialize all variables in one go
session.run(tf.global_variables_initializer())

## initialize a variable on your own
session.run(my_variable.initializer)

## get names of uninitialized variables
print(session.run(tf.report_uinitialized_variables()))

## best way to initialize the variables
v = tf.get_variable("v",shape=(),initializer=tf.zeros_initializer())
w = tf.get_varibale("w",initializer = v.initialized_value()+1)

### Using Variables
在TensorFlow graph当中使用 __tf.Variable__ 值和 __tf.Tensor__ 是一样的，如下示例。  
赋值给一个变量，可以通过 __assign, assign_add__ 方法，或者是 __tf.Variable__ 类当中的一些方法，示例如下。  
大部分TensorFlow optimizers都有特定的ops可以根据类似梯度下降算法来update变量的值。  
由于变量值是可变的，可以通过 __tf.Variable.read_value__ 来获取变量的内容，用例如下

In [27]:
# use the value of tf.Variable
v=tf.get_variable("v",shape=(),initializer=tf.zeros_initializer())
w = v+1  # w is a tf.Tensor which is computed based on the value of v.
           # Any time a variable is used in an expression it gets automatically
           # converted to a tf.Tensor representing its value
with tf.Session() as sess:
    sess.run(v.initialized_value())
    print(w)
    
# assign a value to a variable
v = tf.get_variable("v",shape=(),initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
tf.global_variables_initializer().run()
assignment.run()

# read values of variable
with tf.control.dependencies([assignment]):
    w = v.read_value()# w is guaranteed to reflect v's value after the
                      # assign_add operation.

Tensor("add_4:0", shape=(), dtype=float32)


### Sharing variables
TensorFlow支持两种方式共享变量：
<ul>
    <li>显示的传递<b> tf.Variable </b>对象</li>
    <li>隐式的wrapping <b> tf.Variable</b> 对象通过使用 <b> tf.variable_scope </b>对象</li>
    </ul>  
    <p>
    通过显示的传递变量是非常明显的方法。但是隐式的使用TensorFlow的函数共享变量也是比较便利的，例如 <b>tf.layer, tf.metrics</b> 。
    </p>  
    
    variable scope帮助你通过调用函数隐式的创建和使用变量来控制变量的reuse。他们允许变量通过一个hierarchical and understandable way来命名。用例如下1：
    <p>
    如果说你想要变量可以被共享，你由两个选择：
    <ul>
        <li> 创建一个同名的scope，并且将参数 <b>reuse=True</b></li>
        <li> 利用 <b>scope.reuse_variables()</b> 来触发reuse</li>
        </ul>
        

In [31]:
## 1:write a function to create a convolutional/relu layer:'
def conv_relu(input,kernel_shape,bias_shape):
    #Create variable named "weights"
    weights = tf.get_variable("weights", kernel_shape, 
                              initializer=tf.random_normal_initializer())
    # Create variable named "biases"
    biases = tf.get_variable("biases", bias_shape,
                            initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input,weights,strides=[1,1,1,1],padding='SAME')
    return tf.nn.relu(conv+biases)

input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])
x = conv_relu(input1, kernel_shape=[5,5,32,32],bias_shape=[32])
x = conv_relu(input2, kernel_shape=[5,5,32,32],bias_shape=[32])# This fails
# 第二句fail的原因，因为weights，biases变量已经存在，程序不知道我们当前是想使用
# 原有的变量还是重新建立一个新的变量，所有操作冲突，无法执行。

# 但是通过不同的scopes来调用conv_relu,就会明确我们是想要创建新的变量，那么程序就会work
# this is working when we call conv_relu in different scope
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases"
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])
    
# sharing variables, resue variable
# 使用reuse=True共享变量
with tf.variable_scope("model"):
    output1 = my_image_filter(input1)
with tf.variable_scope("model",reuse=True):
    output2 = my_image_filter(input2)
    
# 使用scope.reuse_variables()共享变量
with tf.variable_scope("model") as scope:
    output1 = my_image_filter(input1)
    scope.reuse_variables()
    output2 = my_image_filger(input2)
    
# 使用特定的字符串来明明scope给人感觉比较危险，所以，可以在其他变量的基础上初始化一个变量
with tf.variable_scope("model") as scope:
    output1 = my_image_filter(input1)
with tf.variable_scope(scope, reuse=True):
    output2 = my_image_filger(input2)